Objectives
 

Project Brief
You work for Spark Funds, an asset management company. Spark Funds wants to make investments in a few companies. The CEO of Spark Funds wants to understand the global trends in investments so that she can take the investment decisions effectively.

 

Business and Data Understanding
Spark Funds has two minor constraints for investments:

It wants to invest between 5 to 15 million USD per round of investment

It wants to invest only in English-speaking countries because of the ease of communication with the companies it would invest in

For your analysis, consider a country to be English speaking only if English is one of the official languages in that country

You may use this list: Click here for a list of countries where English is an official language.

 

These conditions will give you sufficient information for your initial analysis. Before getting to specific questions, let’s understand the problem and the data first.

 

1. What is the strategy?

Spark Funds wants to invest where most other investors are investing. This pattern is often observed among early stage startup investors.

 

2. Where did we get the data from? 

We have taken real investment data from crunchbase.com, so the insights you get may be incredibly useful. For this group project, we have divided the data into the following files:

 

You have to use three main data tables for the entire analysis (available for download on the next page):

 

3. What is Spark Funds’ business objective?

The business objectives and goals of data analysis are pretty straightforward.

Business objective: The objective is to identify the best sectors, countries, and a suitable investment type for making investments. The overall strategy is to invest where others are investing, implying that the 'best' sectors and countries are the ones 'where most investors are investing'.
Goals of data analysis: Your goals are divided into three sub-goals:
Investment type analysis: Comparing the typical investment amounts in the venture, seed, angel, private equity etc. so that Spark Funds can choose the type that is best suited for their strategy.
Country analysis: Identifying the countries which have been the most heavily invested in the past. These will be Spark Funds’ favourites as well.
Sector analysis: Understanding the distribution of investments across the eight main sectors. (Note that we are interested in the eight 'main sectors' provided in the mapping file. The two files — companies and rounds2 — have numerous sub-sector names; hence, you will need to map each sub-sector to its main sector.)
 

4. How do you approach the case study? What are the deliverables?

The entire case study is divided into checkpoints to help you navigate. For each checkpoint, you are advised to fill in the tables into the spreadsheet provided in the download segment. The tables are also mentioned under the 'Results Expected' section after each checkpoint. Since this is the first case study, you have been provided with some additional guidance. Going forward you will be expected to structure and solve the problem by yourself, just like you would be solving problems in real life scenarios.

 

Important Note: All your code has to be submitted in one Jupyter notebook. For every checkpoint, keep writing code in one well-commented Jupyter notebook which you can submit at the end.

In [425]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [426]:
# Load company dataset
# Without encoding the read_csv was giving an error "Unicode Decode Error"
companies_df = pd.read_csv('Data/companies.txt',delimiter='\t',encoding='unicode_escape')

In [427]:
companies_df.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [428]:
# Load funding rounds file, got same error on the Unicode/Decode Error, used recommended encoding.
rounds2_df = pd.read_csv('Data/rounds2.csv',encoding='unicode_escape')

In [429]:
# Quickly inspect the data frame
rounds2_df.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [430]:
rounds2_df.count()

company_permalink          114949
funding_round_permalink    114949
funding_round_type         114949
funding_round_code          31140
funded_at                  114949
raised_amount_usd           94959
dtype: int64

In [431]:
# Load the mapping file for the sector classification
sector_mapping_df = pd.read_csv('Data/mapping.csv')

In [432]:
sector_mapping_df.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [433]:
# Load the country codes file
# source: https://github.com/datasets/country-codes/blob/master/data/country-codes.csv
# columns of interest: country name: official_name_en, Country code: ISO3166-1-Alpha-3
country_df = pd.read_csv('Data/country-codes.csv')

In [434]:
country_df.head()

,FIFA,Dial,ISO3166-1-Alpha-3,MARC,is_independent,ISO3166-1-numeric,GAUL,FIPS,WMO,ISO3166-1-Alpha-2,...,Sub-region Name,official_name_ru,Global Name,Capital,Continent,TLD,Languages,Geoname ID,CLDR display name,EDGAR
0,TPE,886,TWN,ch,Yes,158.0,925,TW,,TW,...,NaN,NaN,NaN,Taipei,AS,.tw,"zh-TW,zh,nan,hak",1668284.0,Taiwan,NaN
1,AFG,93,AFG,af,Yes,4.0,1,AF,AF,AF,...,Southern Asia,Афганистан,World,Kabul,AS,.af,"fa-AF,ps,uz-AF,tk",1149361.0,Afghanistan,B2
2,ALB,355,ALB,aa,Yes,8.0,3,AL,AB,AL,...,Southern Europe,Албания,World,Tirana,EU,.al,"sq,el",783754.0,Albania,B3
3,ALG,213,DZA,ae,Yes,12.0,4,AG,AL,DZ,...,Northern Africa,Алжир,World,Algiers,AF,.dz,ar-DZ,2589581.0,Algeria,B4
4,ASA,1-684,ASM,as,Territory of US,16.0,5,AQ,,AS,...,Polynesia,Американское Самоа,World,Pago Pago,OC,.as,"en-AS,sm,to",5880801.0,American Samoa,B5


In [435]:
country_df[['official_name_en','ISO3166-1-Alpha-3']]

,official_name_en,ISO3166-1-Alpha-3
0,NaN,TWN
1,Afghanistan,AFG
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM
5,Andorra,AND
6,Angola,AGO
7,Anguilla,AIA
8,Antarctica,ATA
9,Antigua and Barbuda,ATG


In [436]:
# Rename the columns for easier analysis in later stages - only country - code & code are required for now
country_df.rename(columns={'official_name_en': 'Name', 'ISO3166-1-Alpha-3': 'Code'}, inplace=True)

In [437]:
country_df[['Code','Name']]

,Code,Name
0,TWN,NaN
1,AFG,Afghanistan
2,ALB,Albania
3,DZA,Algeria
4,ASM,American Samoa
5,AND,Andorra
6,AGO,Angola
7,AIA,Anguilla
8,ATA,Antarctica
9,ATG,Antigua and Barbuda


In [438]:
# Create a new dataframe for the english speaking countries
eng_countries_list = [{'continent':'Asia','country':'India'},
                      {'continent':'Asia','country':'Pakistan'},
                      {'continent':'Asia','country':'Philippines'},
                      {'continent':'Asia','country':'Singapore'},
                      {'continent':'Africa','country':'Botswana'},
                      {'continent':'Africa','country':'Cameroon'},
                      {'continent':'Africa','country':'Ethiopia'},
                      {'continent':'Africa','country':'Eritrea'},
                      {'continent':'Africa','country':'The Gambia'},
                      {'continent':'Africa','country':'Ghana'},
                      {'continent':'Africa','country':'Kenya'},
                      {'continent':'Africa','country':'Lesotho'},
                      {'continent':'Africa','country':'Liberia'},
                      {'continent':'Africa','country':'Malawi'},
                      {'continent':'Africa','country':'Mauritius'},
                      {'continent':'Africa','country':'Namibia'},
                      {'continent':'Africa','country':'Nigeria'},
                      {'continent':'Africa','country':'Rwanda'},
                      {'continent':'Africa','country':'Seychelles'},
                      {'continent':'Africa','country':'Sierra Leone'},
                      {'continent':'Africa','country':'South Africa'},
                      {'continent':'Africa','country':'South Sudan'},
                      {'continent':'Africa','country':'Sudan'},
                      {'continent':'Africa','country':'Swaziland'},
                      {'continent':'Africa','country':'Tanzania'},
                      {'continent':'Africa','country':'Uganda'},
                      {'continent':'Africa','country':'Zambia'},
                      {'continent':'Africa','country':'Zimbabwe'},
                      {'continent':'Americas','country':'Antigua and Barbuda'},
                      {'continent':'Americas','country':'Bahamas'},
                      {'continent':'Americas','country':'Barbados'},
                      {'continent':'Americas','country':'Belize'},
                      {'continent':'Americas','country':'Canada'},
                      {'continent':'Americas','country':'Dominica'},
                      {'continent':'Americas','country':'Grenada'},
                      {'continent':'Americas','country':'Guyana'},
                      {'continent':'Americas','country':'Jamaica'},
                      {'continent':'Americas','country':'Saint Kitts and Nevis'},
                      {'continent':'Americas','country':'Saint Lucia'},
                      {'continent':'Americas','country':'Saint Vincent and the Grenadines'},
                      {'continent':'Americas','country':'Trinidad and Tobago'},
                      {'continent':'Americas','country':'United States of America'},
                      {'continent':'Australia/Oceania','country':'Australia'},
                      {'continent':'Australia/Oceania','country':'Fiji'},
                      {'continent':'Australia/Oceania','country':'Kiribati'},
                      {'continent':'Australia/Oceania','country':'Marshall Islands'},
                      {'continent':'Australia/Oceania','country':'Federated States of Micronesia'},
                      {'continent':'Australia/Oceania','country':'Nauru'},
                      {'continent':'Australia/Oceania','country':'New Zealand'},
                      {'continent':'Australia/Oceania','country':'Palau'},
                      {'continent':'Australia/Oceania','country':'Papua New Guinea'},
                      {'continent':'Australia/Oceania','country':'Samoa'},
                      {'continent':'Australia/Oceania','country':'Solomon Islands'},
                      {'continent':'Australia/Oceania','country':'Tonga'},
                      {'continent':'Australia/Oceania','country':'Tuvalu'},
                      {'continent':'Australia/Oceania','country':'Vanuatu'},
                      {'continent':'Europe','country':'Ireland'},
                      {'continent':'Europe','country':'Malta'},
                      {'continent':'Europe','country':'United Kingdom'}]
eng_countries_df = pd.DataFrame(eng_countries_list)

In [439]:
eng_countries_df

,continent,country
0,Asia,India
1,Asia,Pakistan
2,Asia,Philippines
3,Asia,Singapore
4,Africa,Botswana
5,Africa,Cameroon
6,Africa,Ethiopia
7,Africa,Eritrea
8,Africa,The Gambia
9,Africa,Ghana


In [440]:
# Make sure the rows/columns count matches with the .csv files
companies_df.shape

(66368, 10)

In [441]:
rounds2_df.shape

(114949, 6)

In [442]:
companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [443]:
rounds2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB


In [444]:
sector_mapping_df.shape

(688, 10)

In [445]:
sector_mapping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 10 columns):
category_list                              687 non-null object
Automotive & Sports                        688 non-null int64
Blanks                                     688 non-null int64
Cleantech / Semiconductors                 688 non-null int64
Entertainment                              688 non-null int64
Health                                     688 non-null int64
Manufacturing                              688 non-null int64
News, Search and Messaging                 688 non-null int64
Others                                     688 non-null int64
Social, Finance, Analytics, Advertising    688 non-null int64
dtypes: int64(9), object(1)
memory usage: 53.8+ KB


In [446]:
# Merge countries dataset and english speaking countries dataset
# pd.merge(df_a, df_b, on='subject_id', how='inner')
eng_country_df= pd.merge(eng_countries_df,country_df, left_on='country',right_on='Name',how='left')[['Code','country','continent']]

In [447]:
eng_country_df

,Code,country,continent
0,IND,India,Asia
1,PAK,Pakistan,Asia
2,PHL,Philippines,Asia
3,SGP,Singapore,Asia
4,BWA,Botswana,Africa
5,CMR,Cameroon,Africa
6,ETH,Ethiopia,Africa
7,ERI,Eritrea,Africa
8,NaN,The Gambia,Africa
9,GHA,Ghana,Africa


In [448]:
# update missing country codes
eng_country_df.loc[(eng_country_df.country=='United Kingdom'),'Code']='GBR'
eng_country_df.loc[(eng_country_df.country=='The Gambia'),'Code']='GMB'
eng_country_df.loc[(eng_country_df.country=='Federated States of Micronesia'),'Code']='FSM'
eng_country_df.loc[(eng_country_df.country=='Tanzania'),'Code']='TZA'
eng_country_df.loc[(eng_country_df.country=='Swaziland'),'Code']='SWZ'
# Not sure about the country codes for Federated States of Micronesia & Swaziland

In [449]:
eng_country_df

,Code,country,continent
0,IND,India,Asia
1,PAK,Pakistan,Asia
2,PHL,Philippines,Asia
3,SGP,Singapore,Asia
4,BWA,Botswana,Africa
5,CMR,Cameroon,Africa
6,ETH,Ethiopia,Africa
7,ERI,Eritrea,Africa
8,GMB,The Gambia,Africa
9,GHA,Ghana,Africa


In [450]:
# Find companies that are in English speaking countries
pd.merge(companies_df,eng_country_df, left_on='country_code',right_on='Code', how='left')

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,Code,country,continent
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,IND,India,Asia
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,USA,United States of America,Americas
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,NaN,NaN,NaN
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010,USA,United States of America,Americas
5,/Organization/01Games-Technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN,NaN,NaN,NaN
6,/Organization/0Ndine-Biomedical-Inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997,CAN,Canada,Americas
7,/Organization/0Xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,USA,United States of America,Americas
8,/Organization/1,One Inc.,http://whatis1.com,Mobile,operating,USA,CA,SF Bay Area,San Francisco,01-08-2011,USA,United States of America,Americas
9,/Organization/1-2-3-Listo,"1,2,3 Listo",http://www.123listo.com,E-Commerce,operating,CHL,12,Santiago,Las Condes,01-01-2012,NaN,NaN,NaN


In [451]:
#Find the column wise percentage null count for rounds2 dataframe 
round(100*(rounds2_df.isnull().sum()/len(rounds2_df.index)),2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

In [452]:
#nunique function works on a single column, if we have to take distinct rows on multiple columns, have to use drop_duplicates
#Excel sheet shows 66368, need to verify. 66370 can't be right, only 66368 companies exists
rounds2_df['company_permalink']=rounds2_df['company_permalink'].str.lower()
rounds2_df['company_permalink'].nunique()

66370

In [453]:
# Unique count of companies in Company table.
companies_df['permalink']=companies_df['permalink'].str.lower()
companies_df['permalink'].nunique()

66368

In [454]:
#Are there any companies in the rounds2 file which are not  present in companies ? Answer Y/N.
pd.merge(rounds2_df,companies_df, left_on='company_permalink'
         ,right_on='permalink', how='left')['company_permalink'].isnull().sum(axis=0)

0

In [455]:
rounds2_df

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0
5,/organization/004-technologies,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,24-07-2014,NaN
6,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0
7,/organization/0ndine-biomedical-inc,/funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d,seed,NaN,11-09-2009,43360.0
8,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,NaN,21-12-2009,719491.0
9,/organization/0xdata,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,22-05-2013,3000000.0


In [456]:
# rounds 2 table has data for only valid companies, so left join from rounds2 to company is still valid.
master_frame = pd.merge(rounds2_df,companies_df, left_on='company_permalink'
         ,right_on='permalink', how='left')
master_frame
#master_frame[master_frame['permalink'].isnull()]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
5,/organization/004-technologies,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,24-07-2014,NaN,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010
6,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN
7,/organization/0ndine-biomedical-inc,/funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d,seed,NaN,11-09-2009,43360.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997
8,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,NaN,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997
9,/organization/0xdata,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,22-05-2013,3000000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011


In [457]:
#Drop all the rows which have Null values for raised_amount_usd column - 
#No imputation done for missing values, given that the raised_amount_usd is critical data poitn for analysis
master_frame = master_frame[~np.isnan(master_frame['raised_amount_usd'])]
master_frame

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
6,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN
7,/organization/0ndine-biomedical-inc,/funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d,seed,NaN,11-09-2009,43360.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997
8,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,NaN,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997
9,/organization/0xdata,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,22-05-2013,3000000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011
10,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,09-11-2015,20000000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011
11,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,NaN,03-01-2013,1700000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011


In [458]:
#Find the average amaount raised for each funding type
#investment_type =  master_frame.groupby('funding_round_type').raised_amount_usd.mean().sort_values(ascending = True)
master_frame.groupby('funding_round_type').raised_amount_usd.mean().sort_values(ascending = False).apply(lambda x: '%.2f' % x)
#investment_type - filter the list to show only 5-15M

funding_round_type
post_ipo_debt            168704571.82
post_ipo_equity           82182493.87
secondary_market          79649630.10
private_equity            73308593.03
undisclosed               19242370.23
debt_financing            17043526.02
venture                   11748949.13
grant                      4300576.34
convertible_note           1453438.54
product_crowdfunding       1363131.07
angel                       958694.47
seed                        719818.00
equity_crowdfunding         538368.21
non_equity_assistance       411203.05
Name: raised_amount_usd, dtype: object

In [459]:
master_frame.groupby('funding_round_type').raised_amount_usd.sum().sort_values(ascending = False).apply(lambda x: '%.2f' % x)

funding_round_type
venture                  590126216876.00
private_equity           141925436105.00
debt_financing           113884840886.00
post_ipo_equity           51610606151.00
undisclosed               30441429697.00
post_ipo_debt             25643094917.00
seed                      16989864181.00
grant                      9134424145.00
angel                      4659255123.00
secondary_market           2389488903.00
convertible_note           2103125572.00
equity_crowdfunding         661116163.00
product_crowdfunding        526168593.00
non_equity_assistance        30017823.00
Name: raised_amount_usd, dtype: object

In [475]:
# Create a top 9 data frame for the chosen investment type - venture 
master_frame = master_frame.loc[master_frame.funding_round_type=='venture']
top9 = master_frame.groupby('country_code').raised_amount_usd.sum().sort_values(ascending = False).head(9).apply(lambda x: '%.2f' % x)
top9

country_code
USA    422510842796.00
CHN     39835418773.00
GBR     20245627416.00
IND     14391858718.00
CAN      9583332317.00
FRA      7259536732.00
ISR      6907514579.00
DEU      6346959822.00
JPN      3363676611.00
Name: raised_amount_usd, dtype: object

In [476]:
#filter the master frame for Eng. Speaking countries only.
master_frame= pd.merge(master_frame,eng_country_df, left_on='country_code',right_on='Code', how='left')
master_frame = master_frame.loc[master_frame.Code.notnull()]

In [477]:
master_frame

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,primary_sector,Code,country,continent
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media,IND,India,Asia
3,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,NaN,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997,Biotechnology,CAN,Canada,Americas
4,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,09-11-2015,20000000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
5,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,NaN,03-01-2013,1700000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
6,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,A,19-07-2014,8900000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
7,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,NaN,17-03-2015,5000000.0,/organization/1-mainstream,1 Mainstream,http://www.1mainstream.com,Apps|Cable|Distribution|Software,acquired,USA,CA,SF Bay Area,Cupertino,01-03-2012,Apps,USA,United States of America,Americas
8,/organization/10-minutes-with,/funding-round/0faccbbcc5818dc5326469f13f5a8ac8,venture,A,09-10-2014,4000000.0,/organization/10-minutes-with,10 Minutes With,http://10minuteswith.com,Education,operating,GBR,H9,London,London,01-01-2013,Education,GBR,United Kingdom,Europe
9,/organization/1000memories,/funding-round/502bd0e50c27616995e4bdad24605ef8,venture,A,16-02-2011,2520000.0,/organization/1000memories,1000memories,http://1000memories.com,Curated Web,acquired,USA,CA,SF Bay Area,San Francisco,01-07-2010,Curated Web,USA,United States of America,Americas
10,/organization/1000museums-com,/funding-round/13be128d655076a025221d7fddc90d68,venture,NaN,14-10-2009,265940.0,/organization/1000museums-com,1000museums.com,http://www.1000museums.com,Curated Web,operating,USA,MA,MA - Other,Lenox,01-01-2008,Curated Web,USA,United States of America,Americas
11,/organization/1000museums-com,/funding-round/6aeb32ab3fc05db2b606d7a2466e8c09,venture,NaN,15-01-2014,2906832.0,/organization/1000museums-com,1000museums.com,http://www.1000museums.com,Curated Web,operating,USA,MA,MA - Other,Lenox,01-01-2008,Curated Web,USA,United States of America,Americas


In [478]:
#Top three English speaking countries in terms of investment
top9_df = pd.DataFrame(top9.reset_index())
top9_df

,country_code,raised_amount_usd
0,USA,422510842796.00
1,CHN,39835418773.00
2,GBR,20245627416.00
3,IND,14391858718.00
4,CAN,9583332317.00
5,FRA,7259536732.00
6,ISR,6907514579.00
7,DEU,6346959822.00
8,JPN,3363676611.00


In [479]:
# top 3 countries - English speaking only.
top3 = pd.merge(top9_df,eng_country_df,left_on='country_code',right_on='Code',how='inner').head(3)
top3

,country_code,raised_amount_usd,Code,country,continent
0,USA,422510842796.00,USA,United States of America,Americas
1,GBR,20245627416.00,GBR,United Kingdom,Europe
2,IND,14391858718.00,IND,India,Asia


In [259]:
# Sector Analysis
# Modify sector mapping dataframe category columns to rows.# Use Pandas Melt function
sector_mapping_df1 = pd.melt(sector_mapping_df,id_vars=["category_list"],var_name='main_sector',value_name='flag')
sector_mapping_df1 = sector_mapping_df1[sector_mapping_df1['flag']==1]
sector_mapping_df1

,category_list,main_sector,flag
8,Adventure Travel,Automotive & Sports,1
14,Aerospace,Automotive & Sports,1
45,Auto,Automotive & Sports,1
46,Automated Kiosk,Automotive & Sports,1
47,Automotive,Automotive & Sports,1
57,Bicycles,Automotive & Sports,1
69,Boating Industry,Automotive & Sports,1
87,CAD,Automotive & Sports,1
93,Cars,Automotive & Sports,1
188,Design,Automotive & Sports,1


In [260]:
# Noticed the category list values are having 0 instead of 'na', replacing it to correct the category list values.
sector_mapping_df1['category_list'] = sector_mapping_df1['category_list'].str.replace('0','na')

In [261]:
#drop blank category list from the mapping data frame.
sector_mapping_df1 = sector_mapping_df1[sector_mapping_df1.category_list.notnull()]

In [472]:
sector_mapping_df1

,category_list,main_sector,flag
8,Adventure Travel,Automotive & Sports,1
14,Aerospace,Automotive & Sports,1
45,Auto,Automotive & Sports,1
46,Automated Kiosk,Automotive & Sports,1
47,Automotive,Automotive & Sports,1
57,Bicycles,Automotive & Sports,1
69,Boating Industry,Automotive & Sports,1
87,CAD,Automotive & Sports,1
93,Cars,Automotive & Sports,1
188,Design,Automotive & Sports,1


In [480]:
#Exclude company rounds data where there is no category_list for a company
master_frame = master_frame[master_frame['category_list'].notnull()]
master_frame

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,primary_sector,Code,country,continent
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media,IND,India,Asia
3,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,NaN,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997,Biotechnology,CAN,Canada,Americas
4,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,09-11-2015,20000000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
5,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,NaN,03-01-2013,1700000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
6,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,A,19-07-2014,8900000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
7,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,NaN,17-03-2015,5000000.0,/organization/1-mainstream,1 Mainstream,http://www.1mainstream.com,Apps|Cable|Distribution|Software,acquired,USA,CA,SF Bay Area,Cupertino,01-03-2012,Apps,USA,United States of America,Americas
8,/organization/10-minutes-with,/funding-round/0faccbbcc5818dc5326469f13f5a8ac8,venture,A,09-10-2014,4000000.0,/organization/10-minutes-with,10 Minutes With,http://10minuteswith.com,Education,operating,GBR,H9,London,London,01-01-2013,Education,GBR,United Kingdom,Europe
9,/organization/1000memories,/funding-round/502bd0e50c27616995e4bdad24605ef8,venture,A,16-02-2011,2520000.0,/organization/1000memories,1000memories,http://1000memories.com,Curated Web,acquired,USA,CA,SF Bay Area,San Francisco,01-07-2010,Curated Web,USA,United States of America,Americas
10,/organization/1000museums-com,/funding-round/13be128d655076a025221d7fddc90d68,venture,NaN,14-10-2009,265940.0,/organization/1000museums-com,1000museums.com,http://www.1000museums.com,Curated Web,operating,USA,MA,MA - Other,Lenox,01-01-2008,Curated Web,USA,United States of America,Americas
11,/organization/1000museums-com,/funding-round/6aeb32ab3fc05db2b606d7a2466e8c09,venture,NaN,15-01-2014,2906832.0,/organization/1000museums-com,1000museums.com,http://www.1000museums.com,Curated Web,operating,USA,MA,MA - Other,Lenox,01-01-2008,Curated Web,USA,United States of America,Americas


In [481]:
#Extract a first category_list value (left of |) and add it to the new column primary sector.
master_frame.loc[:,'primary_sector']=master_frame['category_list'].str.split('|', n=1 , expand = True)[0]

Checkpoint 5: Sector Analysis 2
Now you have a data frame with each company’s main sector (main_sector) mapped to it. When we say sector analysis, we refer to one of the eight main sectors.

Also, you know the top three English speaking countries and the most suitable funding type for Spark Funds. Let’s call the three countries 'Country 1', 'Country 2' and 'Country 3' and the funding type 'FT'.

Also, the range of funding preferred by Spark Funds is 5 to 15 million USD.

Now, the aim is to find out the most heavily invested main sectors in each of the three countries (for funding type FT and investments range of 5-15 M USD).

Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:

All the columns of the master_frame along with the primary sector and the main sector

The total number (or count) of investments for each main sector in a separate column

The total amount invested in each main sector in a separate column

Using the three data frames, you can calculate the total number and amount of investments in each main sector.

Result Expected 
Three data frames D1, D2 and D3 

Table 5.1: Based on the analysis of the sectors, which main sectors and countries would you recommend Spark Funds to invest in? Present your conclusions in the presentation. The conclusions are subjective (i.e. there may be no ‘one right answer’), but it should be based on the basic strategy — invest in sectors where most investments are occurring. 

In [482]:
master_frame

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,primary_sector,Code,country,continent
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media,IND,India,Asia
3,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,NaN,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997,Biotechnology,CAN,Canada,Americas
4,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,09-11-2015,20000000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
5,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,NaN,03-01-2013,1700000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
6,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,A,19-07-2014,8900000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011,Analytics,USA,United States of America,Americas
7,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,NaN,17-03-2015,5000000.0,/organization/1-mainstream,1 Mainstream,http://www.1mainstream.com,Apps|Cable|Distribution|Software,acquired,USA,CA,SF Bay Area,Cupertino,01-03-2012,Apps,USA,United States of America,Americas
8,/organization/10-minutes-with,/funding-round/0faccbbcc5818dc5326469f13f5a8ac8,venture,A,09-10-2014,4000000.0,/organization/10-minutes-with,10 Minutes With,http://10minuteswith.com,Education,operating,GBR,H9,London,London,01-01-2013,Education,GBR,United Kingdom,Europe
9,/organization/1000memories,/funding-round/502bd0e50c27616995e4bdad24605ef8,venture,A,16-02-2011,2520000.0,/organization/1000memories,1000memories,http://1000memories.com,Curated Web,acquired,USA,CA,SF Bay Area,San Francisco,01-07-2010,Curated Web,USA,United States of America,Americas
10,/organization/1000museums-com,/funding-round/13be128d655076a025221d7fddc90d68,venture,NaN,14-10-2009,265940.0,/organization/1000museums-com,1000museums.com,http://www.1000museums.com,Curated Web,operating,USA,MA,MA - Other,Lenox,01-01-2008,Curated Web,USA,United States of America,Americas
11,/organization/1000museums-com,/funding-round/6aeb32ab3fc05db2b606d7a2466e8c09,venture,NaN,15-01-2014,2906832.0,/organization/1000museums-com,1000museums.com,http://www.1000museums.com,Curated Web,operating,USA,MA,MA - Other,Lenox,01-01-2008,Curated Web,USA,United States of America,Americas


In [119]:
# Join Master frame category list with the mapping category list, to map primary sector for all funding rounds.
master_frame2= pd.merge(master_frame,sector_mapping_df1, left_on='primary_sector'
         ,right_on='category_list', how='left')

In [265]:
# Apply filter for funding_round_Type "Venture"
master_frame2 = master_frame2[master_frame2.funding_round_type=='venture']

In [267]:
# Invest in companies where they were able to raise between 5 to 15 million $ - we should group by companies to filter the data
master_frame2 = master_frame2[((master_frame2['raised_amount_usd']>=5000000) & 
                                                            (master_frame2['raised_amount_usd']<=15000000))]
master_frame2

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,region,city,founded_at,Code,country,continent,primary_sector,category_list_y,main_sector,flag
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,...,Mumbai,Mumbai,NaN,IND,India,Asia,Media,Media,Entertainment,1.0
12,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,NaN,17-03-2015,5000000.0,/organization/1-mainstream,1 Mainstream,http://www.1mainstream.com,Apps|Cable|Distribution|Software,...,SF Bay Area,Cupertino,01-03-2012,USA,United States of America,Americas,Apps,Apps,"News, Search and Messaging",1.0
56,/organization/128-technology,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,NaN,16-12-2014,11999347.0,/organization/128-technology,128 Technology,http://www.128technology.com/,Service Providers|Technology,...,Boston,Burlington,07-07-2014,USA,United States of America,Americas,Service Providers,Service Providers,Others,1.0
74,/organization/170-systems,/funding-round/b84bb882ca873f5fb96535671981196d,venture,A,16-04-2002,14000000.0,/organization/170-systems,170 Systems,http://www.170systems.com,Software,...,Boston,Bedford,01-01-1990,USA,United States of America,Americas,Software,Software,Others,1.0
83,/organization/1bog,/funding-round/3d2463c9b97f32b3be8747fbd629961f,venture,A,11-02-2010,5000000.0,/organization/1bog,One Block Off the Grid (1BOG),http://1bog.org,Clean Technology|Residential Solar,...,SF Bay Area,San Francisco,01-11-2008,USA,United States of America,Americas,Clean Technology,Clean Technology,Cleantech / Semiconductors,1.0
122,/organization/21diamonds-india,/funding-round/6de7ffef8091ba9f33821f4b861f434a,venture,C,15-11-2012,6369507.0,/organization/21diamonds-india,21Diamonds,http://www.21diamonds.de,E-Commerce,...,New Delhi,Gurgaon,01-06-2012,IND,India,Asia,E-Commerce,E-Commerce,Others,1.0
128,/organization/22nd-century-group,/funding-round/50617d7ea8eaa838e2fd8eb8dfe06054,venture,NaN,18-09-2014,10000000.0,/organization/22nd-century-group,22nd Century Group,http://www.xxiicentury.com,Biotechnology,...,NY - Other,Clarence,01-01-1998,USA,United States of America,Americas,Biotechnology,Biotechnology,Cleantech / Semiconductors,1.0
154,/organization/28msec,/funding-round/ce3a68e4a52892784f1ad085e5d7035a,venture,NaN,28-10-2013,5844811.0,/organization/28msec,28msec,http://www.28msec.com,Cloud Computing|Databases|Software|Web Develop...,...,SF Bay Area,Palo Alto,01-01-2008,USA,United States of America,Americas,Cloud Computing,Cloud Computing,"News, Search and Messaging",1.0
157,/organization/2c2p,/funding-round/fc41599e5ce6654db9217e05e933e5a4,venture,C,27-04-2015,7000000.0,/organization/2c2p,2C2P,http://www.2c2p.com,E-Commerce|Mobile Commerce|Payments|Software,...,NaN,NaN,01-01-2003,SGP,Singapore,Asia,E-Commerce,E-Commerce,Others,1.0
158,/organization/2catalyze,/funding-round/d26e31e33cf217e2ed888eafc54febaf,venture,NaN,25-08-2010,6785876.0,/organization/2catalyze,2Catalyze,NaN,Software,...,Toronto,Toronto,01-01-2008,CAN,Canada,Americas,Software,Software,Others,1.0


In [269]:
#Create three separate dataframes for top three countries and find the most heavily invested main sector in each country
#master_frame2.groupby(['Code','main_sector']).raised_amount_usd.sum().sort_values(ascending = False).apply(lambda x: '%.2f' % x)

Code  main_sector                            
USA   Others                                     9828535193.00
      Social, Finance, Analytics, Advertising    8331086432.00
      Cleantech / Semiconductors                 6387307770.00
      News, Search and Messaging                 4926690815.00
      Health                                     3077665114.00
      Manufacturing                              2798978557.00
      Entertainment                              2101518215.00
      Automotive & Sports                         721333401.00
GBR   Others                                      704816160.00
      Cleantech / Semiconductors                  570845295.00
      Social, Finance, Analytics, Advertising     524276071.00
CAN   Others                                      505127499.00
IND   Others                                      493209507.00
CAN   Cleantech / Semiconductors                  449074893.00
      Social, Finance, Analytics, Advertising     395954075.00
GBR   New

In [485]:
#Create dataframe for top English speaking country (USA) to find sectorwise investment
D1 = master_frame2.loc[master_frame2.country_code=='USA',:]
D1.groupby('main_sector').raised_amount_usd.sum().sort_values(ascending = False).apply(lambda x: '%.2f' % x)

main_sector
Others                                     9828535193.00
Social, Finance, Analytics, Advertising    8331086432.00
Cleantech / Semiconductors                 6387307770.00
News, Search and Messaging                 4926690815.00
Health                                     3077665114.00
Manufacturing                              2798978557.00
Entertainment                              2101518215.00
Automotive & Sports                         721333401.00
Name: raised_amount_usd, dtype: object

In [484]:
#Create dataframe for second English speaking country (GBR) to find sectorwise investment
# Top 2 English speaking country data frame - GBR
D2 = master_frame2.loc[master_frame2.country_code=='GBR',:]
D2.groupby('main_sector').raised_amount_usd.sum().sort_values(ascending = False).apply(lambda x: '%.2f' % x)

main_sector
Others                                     704816160.00
Cleantech / Semiconductors                 570845295.00
Social, Finance, Analytics, Advertising    524276071.00
News, Search and Messaging                 353265942.00
Entertainment                              283493059.00
Manufacturing                              228752591.00
Health                                      79007183.00
Automotive & Sports                         29285381.00
Name: raised_amount_usd, dtype: object

In [489]:
#Create dataframe for third English speaking country (IND) to find sectorwise investment
D3 = master_frame2.loc[master_frame2.country_code=='IND',:]
D3.groupby('main_sector').raised_amount_usd.sum().sort_values(ascending = False).apply(lambda x: '%.2f' % x)

main_sector
Others                                     493209507.00
Social, Finance, Analytics, Advertising    264993500.00
News, Search and Messaging                 197268770.00
Entertainment                              160430000.00
Manufacturing                              148400000.00
Health                                     140740000.00
Automotive & Sports                         59900000.00
Cleantech / Semiconductors                  56330000.00
Name: raised_amount_usd, dtype: object

In [486]:
#Count the total number of investments in top english speaking country
D1.shape

(4781, 23)

In [487]:
#Count the total number of investments in second english speaking country
D2.shape

(349, 23)

In [490]:
#Count the total number of investments in third english speaking country
D3.shape

(185, 23)

In [491]:
#Total amount of investment in Top English speaking country
D1.raised_amount_usd.sum()

38293926264.0

In [284]:
#Total amount of investment in Second English speaking country
D2.raised_amount_usd.sum()

2787592073.0

In [492]:
#Total amount of investment in Third English speaking country
D3.raised_amount_usd.sum()

1521271777.0

In [493]:
#Total number of investment in the Topmost sector in Top English speaking country
D1.loc[D1.main_sector=='Others'].shape

(1242, 23)

In [494]:
#Total number of investment in the Topmost sector in Second English speaking country
D2.loc[D2.main_sector=='Others'].shape

(88, 23)

In [495]:
#Total number of investment in the Topmost sector in Third English speaking country
D3.loc[D3.main_sector=='Others'].shape

(60, 23)

In [496]:
#Total number of investment in the Second Topmost sector in Top English speaking country
D1.loc[D1.main_sector=='Others'].shape

(1242, 23)

In [497]:
#Total number of investment in the Second Topmost sector in Second English speaking country
D2.loc[D2.main_sector=='Cleantech / Semiconductors'].shape

(67, 23)

In [498]:
#Total number of investment in the Second Topmost sector in Third English speaking country
D3.loc[D3.main_sector=='Social, Finance, Analytics, Advertising'].shape

(32, 23)

In [499]:
#Total number of Investments for each main sector for Top English speaking country
D1.loc[:,'No_Of_Investment']=D1.groupby('main_sector')["Code"].transform('count')
#top_sector.sort_values(ascending=False)
top_sector=D1.drop_duplicates(subset=['main_sector','No_Of_Investment'])
top_sector[['main_sector','No_Of_Investment']].sort_values(by='No_Of_Investment',ascending=False).head(3)

,main_sector,No_Of_Investment
56,Others,1242.0
216,"Social, Finance, Analytics, Advertising",1059.0
83,Cleantech / Semiconductors,765.0


In [500]:
#Total number of Investments for each main sector for 2nd Top English speaking country
D2.loc[:,'No_Of_Investment']=D2.groupby('main_sector')["Code"].transform('count')
#top_sector.sort_values(ascending=False)
top_sector=D2.drop_duplicates(subset=['main_sector','No_Of_Investment'])
top_sector[['main_sector','No_Of_Investment']].sort_values(by='No_Of_Investment',ascending=False).head(3)

,main_sector,No_Of_Investment
3055,Others,88.0
222,"Social, Finance, Analytics, Advertising",70.0
632,Cleantech / Semiconductors,67.0


In [501]:
#Total number of Investments for each main sector for 3rd Top English speaking country
D3.loc[:,'No_Of_Investment']=D3.groupby('main_sector')["Code"].transform('count')
#top_sector.sort_values(ascending=False)
top_sector=D3.drop_duplicates(subset=['main_sector','No_Of_Investment'])
top_sector[['main_sector','No_Of_Investment']].sort_values(by='No_Of_Investment',ascending=False).head(3)

,main_sector,No_Of_Investment
122,Others,60
562,"Social, Finance, Analytics, Advertising",32
6617,"News, Search and Messaging",27


In [365]:
#For point 3 (top sector count-wise), which company received the highest investment?
D1.loc[D1['main_sector']=='Others'].groupby('permalink').Code.count().sort_values(ascending = False)
#Multiple companies matches with the highest investment.

permalink
/organization/digital-fuel                     2
/organization/scan                             2
/organization/interactive-tko                  2
/organization/emotive-communications           2
/organization/eat-club                         2
/organization/brabeion-software                2
/organization/imemories                        2
/organization/apex-learning                    2
/organization/anthology-solutions              2
/organization/newmerix                         2
/organization/cista-system                     2
/organization/lumoback                         2
/organization/livevox                          2
/organization/sychron-advanced-technologies    2
/organization/rhone-apparel                    2
/organization/filmloop                         2
/organization/liquid-engines                   2
/organization/the-american-academy             2
/organization/alvenda-inc                      2
/organization/vizerra                          2
/organizat

In [311]:
#Total Amount Invested for each main sector for Top English speaking country
D1.loc[:,'Amount_Invested']=D1.groupby('main_sector')["raised_amount_usd"].transform('sum').apply(lambda x: '%.2f' % x)

,Code,main_sector,No_Of_Investment
54085,USA,NaN,nan
3801,USA,NaN,nan
70530,USA,NaN,nan
49372,USA,NaN,nan
6465,USA,NaN,nan
42261,USA,NaN,nan
41700,USA,NaN,nan
41686,USA,NaN,nan
53566,USA,NaN,nan
41783,USA,NaN,nan


In [502]:
#Total number of Investments for each main sector for second English speaking country
D2.loc[:,'No_Of_Investment']=D2.groupby('main_sector')["raised_amount_usd"].transform('count').apply(lambda x: '%.2f' % x)
D2

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,city,founded_at,Code,country,continent,primary_sector,category_list_y,main_sector,flag,No_Of_Investment
222,/organization/365scores,/funding-round/48212f931f542fdef78810bc87aef086,venture,B,29-09-2014,5500000.0,/organization/365scores,365Scores,http://biz.365scores.com,Android|Apps|iPhone|Mobile|Sports,...,London,01-01-2008,GBR,United Kingdom,Europe,Android,Android,"Social, Finance, Analytics, Advertising",1.0,70.00
632,/organization/abcodia,/funding-round/3d20c23d203134ed86c0d1b2bec288b2,venture,B,18-05-2015,8259067.0,/organization/abcodia,Abcodia,http://abcodia.com,Biotechnology,...,London,01-01-2010,GBR,United Kingdom,Europe,Biotechnology,Biotechnology,Cleantech / Semiconductors,1.0,67.00
785,/organization/acal-enterprise-solutions,/funding-round/0f9a693d9686330c5c2724215e0048e2,venture,NaN,03-06-2014,10720178.0,/organization/acal-enterprise-solutions,Acal Enterprise Solutions,http://acalenterprisesolutions.com,Information Technology,...,Nottingham,01-01-1990,GBR,United Kingdom,Europe,Information Technology,Information Technology,"Social, Finance, Analytics, Advertising",1.0,70.00
840,/organization/accent-media-ltd,/funding-round/9dc643fa45031a46ffcfaa061d94e3e3,venture,NaN,01-07-2013,8300000.0,/organization/accent-media-ltd,Accent Media Limited,http://accent.media,Domains|Internet|Ticketing,...,London,21-02-2012,GBR,United Kingdom,Europe,Domains,Domains,"News, Search and Messaging",1.0,45.00
960,/organization/accumuli-security,/funding-round/f7b76bf555e5619fbe259f697f5fd97e,venture,NaN,01-11-2010,8033458.0,/organization/accumuli-security,Accumuli Security,http://www.accumuli.com,Content|Events|Security|Software|Training,...,Basingstoke,01-01-1996,GBR,United Kingdom,Europe,Content,Content,Entertainment,1.0,36.00
961,/organization/accunostics,/funding-round/dee5892854258f2cd7cf0ec144080ca1,venture,NaN,15-09-2011,6321095.0,/organization/accunostics,AccuNostics,http://www.accunostics.co.uk,Health Care|Health Diagnostics,...,Forres,01-01-2008,GBR,United Kingdom,Europe,Health Care,Health Care,Health,1.0,10.00
1431,/organization/adbrain,/funding-round/71f1b5903874817292d67e1ac6aa66b2,venture,A,11-03-2014,7500000.0,/organization/adbrain,Adbrain,http://adbrain.com,Advertising|Enterprise Software|Marketing Auto...,...,London,01-01-2012,GBR,United Kingdom,Europe,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,70.00
1565,/organization/adinsight,/funding-round/5bcca548aeccd5acdea0d0edb086db8d,venture,B,07-05-2014,6772812.0,/organization/adinsight,ResponseTap,http://www.responsetap.com,Advertising|Enterprise Software|SaaS,...,Manchester,01-06-2008,GBR,United Kingdom,Europe,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,70.00
1583,/organization/adjug,/funding-round/83653befbf3c473ec97a1617479448c8,venture,B,01-03-2008,6500000.0,/organization/adjug,Adjug,http://www.adjug.com,Advertising|Internet|Marketplaces|Publishing,...,London,01-01-2007,GBR,United Kingdom,Europe,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,70.00
1926,/organization/aea-technology,/funding-round/10be04d48813f3a5cf49160d400d5582,venture,A,28-08-2008,6500000.0,/organization/aea-technology,AEA Technology,http://www.aeat.co.uk,Clean Technology,...,NaN,NaN,GBR,United Kingdom,Europe,Clean Technology,Clean Technology,Cleantech / Semiconductors,1.0,67.00


In [503]:
#Total Amount Invested for each main sector for second English speaking country
D2.loc[:,'Amount_Invested']=D2.groupby('main_sector')["raised_amount_usd"].transform('sum').apply(lambda x: '%.2f' % x)
D2

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,founded_at,Code,country,continent,primary_sector,category_list_y,main_sector,flag,No_Of_Investment,Amount_Invested
222,/organization/365scores,/funding-round/48212f931f542fdef78810bc87aef086,venture,B,29-09-2014,5500000.0,/organization/365scores,365Scores,http://biz.365scores.com,Android|Apps|iPhone|Mobile|Sports,...,01-01-2008,GBR,United Kingdom,Europe,Android,Android,"Social, Finance, Analytics, Advertising",1.0,70.00,524276071.00
632,/organization/abcodia,/funding-round/3d20c23d203134ed86c0d1b2bec288b2,venture,B,18-05-2015,8259067.0,/organization/abcodia,Abcodia,http://abcodia.com,Biotechnology,...,01-01-2010,GBR,United Kingdom,Europe,Biotechnology,Biotechnology,Cleantech / Semiconductors,1.0,67.00,570845295.00
785,/organization/acal-enterprise-solutions,/funding-round/0f9a693d9686330c5c2724215e0048e2,venture,NaN,03-06-2014,10720178.0,/organization/acal-enterprise-solutions,Acal Enterprise Solutions,http://acalenterprisesolutions.com,Information Technology,...,01-01-1990,GBR,United Kingdom,Europe,Information Technology,Information Technology,"Social, Finance, Analytics, Advertising",1.0,70.00,524276071.00
840,/organization/accent-media-ltd,/funding-round/9dc643fa45031a46ffcfaa061d94e3e3,venture,NaN,01-07-2013,8300000.0,/organization/accent-media-ltd,Accent Media Limited,http://accent.media,Domains|Internet|Ticketing,...,21-02-2012,GBR,United Kingdom,Europe,Domains,Domains,"News, Search and Messaging",1.0,45.00,353265942.00
960,/organization/accumuli-security,/funding-round/f7b76bf555e5619fbe259f697f5fd97e,venture,NaN,01-11-2010,8033458.0,/organization/accumuli-security,Accumuli Security,http://www.accumuli.com,Content|Events|Security|Software|Training,...,01-01-1996,GBR,United Kingdom,Europe,Content,Content,Entertainment,1.0,36.00,283493059.00
961,/organization/accunostics,/funding-round/dee5892854258f2cd7cf0ec144080ca1,venture,NaN,15-09-2011,6321095.0,/organization/accunostics,AccuNostics,http://www.accunostics.co.uk,Health Care|Health Diagnostics,...,01-01-2008,GBR,United Kingdom,Europe,Health Care,Health Care,Health,1.0,10.00,79007183.00
1431,/organization/adbrain,/funding-round/71f1b5903874817292d67e1ac6aa66b2,venture,A,11-03-2014,7500000.0,/organization/adbrain,Adbrain,http://adbrain.com,Advertising|Enterprise Software|Marketing Auto...,...,01-01-2012,GBR,United Kingdom,Europe,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,70.00,524276071.00
1565,/organization/adinsight,/funding-round/5bcca548aeccd5acdea0d0edb086db8d,venture,B,07-05-2014,6772812.0,/organization/adinsight,ResponseTap,http://www.responsetap.com,Advertising|Enterprise Software|SaaS,...,01-06-2008,GBR,United Kingdom,Europe,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,70.00,524276071.00
1583,/organization/adjug,/funding-round/83653befbf3c473ec97a1617479448c8,venture,B,01-03-2008,6500000.0,/organization/adjug,Adjug,http://www.adjug.com,Advertising|Internet|Marketplaces|Publishing,...,01-01-2007,GBR,United Kingdom,Europe,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,70.00,524276071.00
1926,/organization/aea-technology,/funding-round/10be04d48813f3a5cf49160d400d5582,venture,A,28-08-2008,6500000.0,/organization/aea-technology,AEA Technology,http://www.aeat.co.uk,Clean Technology,...,NaN,GBR,United Kingdom,Europe,Clean Technology,Clean Technology,Cleantech / Semiconductors,1.0,67.00,570845295.00


In [504]:
#Total number of Investments for each main sector for Third English speaking country
D3.loc[:,'No_Of_Investment']=D3.groupby('main_sector')["raised_amount_usd"].transform('count').apply(lambda x: '%.2f' % x)
D3

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,city,founded_at,Code,country,continent,primary_sector,category_list_y,main_sector,flag,No_Of_Investment
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,...,Mumbai,NaN,IND,India,Asia,Media,Media,Entertainment,1.0,19.00
122,/organization/21diamonds-india,/funding-round/6de7ffef8091ba9f33821f4b861f434a,venture,C,15-11-2012,6369507.0,/organization/21diamonds-india,21Diamonds,http://www.21diamonds.de,E-Commerce,...,Gurgaon,01-06-2012,IND,India,Asia,E-Commerce,E-Commerce,Others,1.0,60.00
562,/organization/a-little-world,/funding-round/18d98f82ed392b1609975b81f3e8b3fb,venture,B,09-09-2008,6410000.0,/organization/a-little-world,A LITTLE WORLD,http://alittleworld.com,Finance,...,Mumbai,02-03-2000,IND,India,Asia,Finance,Finance,"Social, Finance, Analytics, Advertising",1.0,32.00
1591,/organization/adlabs-imagica,/funding-round/508d3c83daaae9fda3ba6f9682c78f6c,venture,NaN,28-10-2014,8180000.0,/organization/adlabs-imagica,Adlabs Imagica,http://www.adlabsimagica.com,Entertainment|Tourism,...,Khopoli,NaN,IND,India,Asia,Entertainment,Entertainment,Entertainment,1.0,19.00
2220,/organization/agile,/funding-round/cd3dd1c98ce9d0f632d8752163941674,venture,A,01-05-2011,5740000.0,/organization/agile,Agile,http://www.agile-ft.com,Finance|Finance Technology|FinTech|Insurance,...,Mumbai,NaN,IND,India,Asia,Finance,Finance,"Social, Finance, Analytics, Advertising",1.0,32.00
3281,/organization/amagi-media-labs,/funding-round/f244a91cc714317f6fbbc80dcc1d5135,venture,A,17-06-2013,5500000.0,/organization/amagi-media-labs,Amagi Media Labs,http://amagi.com,Advertising,...,Bangalore,01-01-2008,IND,India,Asia,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,32.00
3470,/organization/ameyo,/funding-round/81b50a403d5d2293715fe9b0ce4db5d3,venture,A,03-07-2015,5000000.0,/organization/ameyo,Ameyo,http://www.ameyo.com/,Software,...,Gurgaon,01-01-2003,IND,India,Asia,Software,Software,Others,1.0,60.00
3834,/organization/annapurna-microfinace,/funding-round/3f03bc9fea4ae59b1ce8c86a0782107e,venture,B,26-03-2014,5000000.0,/organization/annapurna-microfinace,Annapurna Microfinace,http://ampl.net.in,Finance,...,Bhubaneswar,01-01-2009,IND,India,Asia,Finance,Finance,"Social, Finance, Analytics, Advertising",1.0,32.00
4303,/organization/applabs,/funding-round/29152fc812013ef482c93f6dd47f58ff,venture,NaN,29-06-2006,10000000.0,/organization/applabs,AppLabs,http://www.applabs.com,Software,...,Hyderabad,01-01-2001,IND,India,Asia,Software,Software,Others,1.0,60.00
5520,/organization/ather-energy,/funding-round/a3782f52b69e60629bcf7866ca8b1eca,venture,A,29-05-2015,12000000.0,/organization/ather-energy,Ather Energy,http://www.atherenergy.com,Automotive|Electric Vehicles,...,Bangalore,01-04-2013,IND,India,Asia,Automotive,Automotive,Automotive & Sports,1.0,6.00


In [505]:
#Total Amount Invested for each main sector for Third English speaking country
D3.loc[:,'Amount_Invested']=D3.groupby('main_sector')["raised_amount_usd"].transform('sum').apply(lambda x: '%.2f' % x)
D3

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,founded_at,Code,country,continent,primary_sector,category_list_y,main_sector,flag,No_Of_Investment,Amount_Invested
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,...,NaN,IND,India,Asia,Media,Media,Entertainment,1.0,19.00,160430000.00
122,/organization/21diamonds-india,/funding-round/6de7ffef8091ba9f33821f4b861f434a,venture,C,15-11-2012,6369507.0,/organization/21diamonds-india,21Diamonds,http://www.21diamonds.de,E-Commerce,...,01-06-2012,IND,India,Asia,E-Commerce,E-Commerce,Others,1.0,60.00,493209507.00
562,/organization/a-little-world,/funding-round/18d98f82ed392b1609975b81f3e8b3fb,venture,B,09-09-2008,6410000.0,/organization/a-little-world,A LITTLE WORLD,http://alittleworld.com,Finance,...,02-03-2000,IND,India,Asia,Finance,Finance,"Social, Finance, Analytics, Advertising",1.0,32.00,264993500.00
1591,/organization/adlabs-imagica,/funding-round/508d3c83daaae9fda3ba6f9682c78f6c,venture,NaN,28-10-2014,8180000.0,/organization/adlabs-imagica,Adlabs Imagica,http://www.adlabsimagica.com,Entertainment|Tourism,...,NaN,IND,India,Asia,Entertainment,Entertainment,Entertainment,1.0,19.00,160430000.00
2220,/organization/agile,/funding-round/cd3dd1c98ce9d0f632d8752163941674,venture,A,01-05-2011,5740000.0,/organization/agile,Agile,http://www.agile-ft.com,Finance|Finance Technology|FinTech|Insurance,...,NaN,IND,India,Asia,Finance,Finance,"Social, Finance, Analytics, Advertising",1.0,32.00,264993500.00
3281,/organization/amagi-media-labs,/funding-round/f244a91cc714317f6fbbc80dcc1d5135,venture,A,17-06-2013,5500000.0,/organization/amagi-media-labs,Amagi Media Labs,http://amagi.com,Advertising,...,01-01-2008,IND,India,Asia,Advertising,Advertising,"Social, Finance, Analytics, Advertising",1.0,32.00,264993500.00
3470,/organization/ameyo,/funding-round/81b50a403d5d2293715fe9b0ce4db5d3,venture,A,03-07-2015,5000000.0,/organization/ameyo,Ameyo,http://www.ameyo.com/,Software,...,01-01-2003,IND,India,Asia,Software,Software,Others,1.0,60.00,493209507.00
3834,/organization/annapurna-microfinace,/funding-round/3f03bc9fea4ae59b1ce8c86a0782107e,venture,B,26-03-2014,5000000.0,/organization/annapurna-microfinace,Annapurna Microfinace,http://ampl.net.in,Finance,...,01-01-2009,IND,India,Asia,Finance,Finance,"Social, Finance, Analytics, Advertising",1.0,32.00,264993500.00
4303,/organization/applabs,/funding-round/29152fc812013ef482c93f6dd47f58ff,venture,NaN,29-06-2006,10000000.0,/organization/applabs,AppLabs,http://www.applabs.com,Software,...,01-01-2001,IND,India,Asia,Software,Software,Others,1.0,60.00,493209507.00
5520,/organization/ather-energy,/funding-round/a3782f52b69e60629bcf7866ca8b1eca,venture,A,29-05-2015,12000000.0,/organization/ather-energy,Ather Energy,http://www.atherenergy.com,Automotive|Electric Vehicles,...,01-04-2013,IND,India,Asia,Automotive,Automotive,Automotive & Sports,1.0,6.00,59900000.00
